In [ ]:
# !nvidia-smi

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import sys
# module_path="D:\\Tony\\SRTP\\WheatMappingModel"
# sys.path.append("D:\\Tony\\SRTP\\WheatMappingModel") 
sys.path.append("/content/drive/MyDrive/WheatMappingModel")
from logger import PrettyLogger
from helper import WMMhelper

In [ ]:
logger = PrettyLogger()
helper = WMMhelper()
DEVICE = torch.device("cuda:0")

In [ ]:
path_template = "/content/drive/MyDrive/SRTP_Datasets/DatasetsForWMM"
def get_path(years, locations):
    x_path_list = []
    y_path_list = []
    for location in locations:
        for year in years:
            for i in range(1, 201):
                x_path = path_template + "/" + str(location) + "/" + str(year) + "/" + "x"+str(i)+".npy"
                y_path = path_template + "/" + str(location) + "/" + str(year) + "/" + "y"+str(i)+".npy"
                x_path_list.append(x_path)
                y_path_list.append(y_path)
    return x_path_list, y_path_list

In [ ]:
x_train_path, y_train_path = get_path([2015,2016,2017,2018],[20191])
x_test_path, y_test_path = get_path([2019],[20191])
x_train = helper.input_x(x_train_path)
y_train = helper.input_y(y_train_path)
x_test = helper.input_x(x_test_path)
y_test = helper.input_y(y_test_path)
N_num = (y_train == 0).sum()
P_num = (y_train == 1).sum()
cur_pos_weight = N_num/P_num
helper.criterion = nn.BCEWithLogitsLoss(reduction='mean', pos_weight= cur_pos_weight)
print(cur_pos_weight)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# x_train, x_test = helper.normalize(x_train, x_test)
# print(x_train.shape)

In [ ]:
train_dataloader = helper.make_dataloader(x_train, y_train)
test_dataloader = helper.make_dataloader(x_test, y_test)
print(train_dataloader)
print(len(train_dataloader.dataset))

In [ ]:
net = helper.build_model()
net.to(DEVICE)
helper.train_model(net, train_dataloader, test_dataloader, DEVICE, logger)